In [1]:
# 1. Import Plotly
import plotly.graph_objects as go
import pandas as pd
import glob
from pathlib import Path

In [2]:
# find relevant data files
csv_files = glob.glob("../logs/gyr*.csv")
print(f"found {len(csv_files)} files: ")
for file in csv_files:
    print(f" - {file}")

# load the most recent file
latest_file = max(csv_files, key=lambda f: Path(f).stat().st_mtime)
df = pd.read_csv(latest_file)
print(f"Loaded {len(df)} data points")

found 1 files: 
 - ../logs/gyro.csv
Loaded 3045 data points


In [3]:
# check data structure
print("data shape:", df.shape);
print("\nColumns:", df.columns.tolist())
print("\nFirst few rows:")
df.head()

data shape: (3045, 5)

Columns: ['timestamp_ns', 'timestamp_sec', 'gyro_x_dps', 'gyro_y_dps', 'gyro_z_dps']

First few rows:


,timestamp_ns,timestamp_sec,gyro_x_dps,gyro_y_dps,gyro_z_dps
0,13400130903642,13400.130904,0.17500,-2.08250,-5.59125
1,13400138317393,13400.138317,0.27125,-1.63625,-5.11000
2,13400145517803,13400.145518,0.27125,-1.89000,-5.26750
3,13400152025355,13400.152025,0.61250,-1.89000,-5.26750
4,13400159267652,13400.159268,0.28000,-2.17000,-5.53000


In [4]:
# compute time jitter in seconds
df['time_jitter'] = df['timestamp_ns'].diff() / 1e6    
df['time_jitter'].fillna(0, inplace=True)
print("\nFirst few rows with time_jitter:")

# plot time jitter
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['time_jitter'], mode='lines+markers', name='Time Jitter (s)'))
fig.update_layout(title='Gyroscope Time Jitter', xaxis_title='Sample Index', yaxis_title='Time Jitter (ms)')
fig.show()


First few rows with time_jitter:


In [5]:
# plot gyroscope data
df['time'] = (df['timestamp_ns'] - df['timestamp_ns'].iloc[0]) / 1e9  # convert to seconds
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['time'], y=df['gyro_x_dps'], mode='lines', name='Gyro X'))
fig.add_trace(go.Scatter(x=df['time'], y=df['gyro_y_dps'], mode='lines', name='Gyro Y'))
fig.add_trace(go.Scatter(x=df['time'], y=df['gyro_z_dps'], mode='lines', name='Gyro Z'))
fig.update_layout(title='Gyroscope Data', xaxis_title='Time (s)', yaxis_title='Angular Velocity (dps)')
fig.show()

In [6]:
# plot delta gyro data
df['delta_gyro_x'] = df['gyro_x_dps'].diff().fillna(0)
df['delta_gyro_y'] = df['gyro_y_dps'].diff().fillna(0)
df['delta_gyro_z'] = df['gyro_z_dps'].diff().fillna(0)  
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['time'], y=df['delta_gyro_x'], mode='lines', name='Delta Gyro X'))
fig.add_trace(go.Scatter(x=df['time'], y=df['delta_gyro_y'], mode='lines', name='Delta Gyro Y'))
fig.add_trace(go.Scatter(x=df['time'], y=df['delta_gyro_z'], mode='lines', name='Delta Gyro Z'))
fig.update_layout(title='Delta Gyroscope Data', xaxis_title='Time (s)', yaxis_title='Delta Angular Velocity (dps)')
fig.show()  